This tutorial will teach you the basics of data transfer using the Streaming Protocol for the Exchange of Astronomical Data (SPEAD). We will use the Spead2 library for Python, which has documentation available at [this link](https://spead2.readthedocs.io/en/latest). We need both a sender and a receiver to transfer data between machines;  in this part, we will implement a basic sender. After installing Spead2, import the following in your Python source file.

In [ ]:
import spead2
import spead2.send
import numpy as np

We need a `ThreadPool` instance in order to instantiate a stream. A `ThreadPool` is a collection of C threads, which allows the `Stream` class to send and receive data.

In [ ]:
thread_pool = spead2.ThreadPool()

The `spead2.send.StreamConfig` lets a user specify parameters for a `Stream` instance. The only thing we are going to change from the default value is the send rate, which we will set to `1e7`. The other defaults are: 
    `max_packet_size=1472`, 
    `rate=0.0`, 
    `burst_size=65535`, and 
    `max_heaps=4`. 
For our purposes, we can ignore these optional parameters. If you would like to dive deeper, however, you can read more about the [StreamConfig](https://spead2.readthedocs.io/en/latest/py-send.html#spead2.send.StreamConfig) class.

In [ ]:
stream_config = spead2.send.StreamConfig(rate=1e7)

We can now instantiate a `UdpStream` which we will use to send Spead data. We must pass in the `ThreadPool` we created earlier, the Ipv4 address and the port we wish to send data to, and finally, the `StreamConfig` object we just created. 

In [ ]:
stream = spead2.send.UdpStream(thread_pool, "10.0.1.112", 8888, stream_config)

The Spead protocol states that to send data, we create `Item` objects, which are accumulated together into `ItemGroups`, and finally chunked into manageable `Heaps` to be sent across the network. We now create a `spead2.send.ItemGroup`, to which we will add our data in the form of `Item`s. The `ItemGroup` will then expose some methods which allow us to easily create `Heap`s which we can send over our stream. 

In [ ]:
item_group = spead2.send.ItemGroup()

To add an `Item` to our `ItemGroup`, we use the `ItemGroup.add_item` method. The arguments to this method let us specify a description of the item we are adding to the `ItemGroup`.

In [ ]:
# this method adds an item descriptor to item_group, and returns the item to us
item = item_group.add_item( 0x1000, "number",   "just some large number", (),         dtype=np.uint32)
#                          item ID  item name   item description          itemShape   item data type

This method returned an `Item` instance. We have only provided a description of this item to the `ItemGroup`, but we have not set its value yet. Let's do that now. The `ItemGroup` is keeping an eye on any changes we make to this `Item` instance, and so changing its value will automatically update our `ItemGroup`.

In [ ]:
item.value = 0xDEADBEEF

There are [some things I did not cover](https://spead2.readthedocs.io/en/latest/py-items.html#spead2.ItemGroup) about `ItemGroup`s, but these are the basics. 

We now instantiate a `HeapGenerator` object, which allows us to chunk our `ItemGroup` into heaps which we can then send over the network. A `HeapGenerator` will track the `ItemGroup` it is instantiated with, meaning that if you update your `ItemGroup` by adding anything to it or changing the values of any items within it, the `HeapGenerator` will include these changes in the next heap you generate. The `HeapGenerator` class is complex and has some optional parameters that you [may want to read about](https://spead2.readthedocs.io/en/latest/py-send.html?highlight=HeapGenerator#spead2.send.HeapGenerator).

In [ ]:
heap_generator = spead2.send.HeapGenerator(item_group)

To send a heap with our `HeapGenerator` and `UdpStream` is pretty straightforward. Take a look:

In [ ]:
stream.send_heap(heap_generator.get_heap())

Once we are done sending all of our data and would like to let the receiver know that it can stop listening, we can send an end-of-heap marker. If you are using the Spead2 receiver from the other part of this tutorial, this will cause the receiver to stop listening for data and begin to print out what is has received. 

In [ ]:
stream.send_heap(heap_generator.get_end())

That's it for some simple sending. Now that you know the basics of the Spead2 Sending API, you can head over to [this link](https://spead2.readthedocs.io/en/latest/) to learn more about the advanced features the library has to offer.